<a href="https://colab.research.google.com/github/shatinz/tripreports/blob/master/tripreport2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers transformers accelerate
!pip install generativeai
!pip install pandas
!pip install google-generativeai

In [2]:
#1.extracting links
import requests
from bs4 import BeautifulSoup

url = "https://www.erowid.org/experiences/exp.cgi?S1=18"
response = requests.get(url, verify=False)  # Disable SSL verification
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tr elements with class "exp-list-row"
rows = soup.find_all('tr', class_='exp-list-row')

# Extract href from the <a> tag within each row
links = []
for row in rows:
    link_tag = row.find('a')
    if link_tag and 'href' in link_tag.attrs:
        links.append("https://www.erowid.org/experiences/" + link_tag['href'])

# Print the extracted links
for link in links:
    print(link)

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://www.erowid.org/experiences/exp.php?ID=118252
https://www.erowid.org/experiences/exp.php?ID=112397
https://www.erowid.org/experiences/exp.php?ID=96444
https://www.erowid.org/experiences/exp.php?ID=64048
https://www.erowid.org/experiences/exp.php?ID=62835
https://www.erowid.org/experiences/exp.php?ID=34866
https://www.erowid.org/experiences/exp.php?ID=113441
https://www.erowid.org/experiences/exp.php?ID=113133
https://www.erowid.org/experiences/exp.php?ID=99695
https://www.erowid.org/experiences/exp.php?ID=94632
https://www.erowid.org/experiences/exp.php?ID=103790
https://www.erowid.org/experiences/exp.php?ID=96614
https://www.erowid.org/experiences/exp.php?ID=106677
https://www.erowid.org/experiences/exp.php?ID=89887
https://www.erowid.org/experiences/exp.php?ID=98866
https://www.erowid.org/experiences/exp.php?ID=101485
https://www.erowid.org/experiences/exp.php?ID=101541
https://www.erowid.org/experiences/exp.php?ID=93315
https://www.erowid.org/experiences/exp.php?ID=96521
http

In [4]:
#extracting html of tables
import requests
from bs4 import BeautifulSoup
from IPython.display import display

# Assuming 'links' list from the previous cell is available

extracted_data = []

for link in links:
    try:
        response = requests.get(link, verify=False)
        soup = BeautifulSoup(response.content, 'html.parser')

        substance_div = soup.find('div', class_='substance')
        substance = substance_div.text.strip() if substance_div else None

        dosechart_table = soup.find('table', class_='dosechart')
        dosechart = str(dosechart_table) if dosechart_table else None

        bodyweight_table = soup.find('table', class_='bodyweight')
        bodyweight = str(bodyweight_table) if bodyweight_table else None

        footdata_table = soup.find('table', class_='footdata')
        footdata = str(footdata_table) if footdata_table else None

        extracted_data.append({
            'link': link,
            'substance': substance,
            'dosechart': dosechart,
            'bodyweight': bodyweight,
            'footdata': footdata
        })

    except Exception as e:
        print(f"Error processing link {link}: {e}")

# You can now process the extracted_data list, for example, print it
display(extracted_data)

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097

[{'link': 'https://www.erowid.org/experiences/exp.php?ID=118252',
  'substance': 'DMT Fumarate & Harmala Alkaloids',
  'dosechart': '<table class="dosechart">\n<tr>\n<td align="right" class="dosechart-time">DOSE:<br/> T+ 0:00</td>\n<td align="center" class="dosechart-amount">1 - 3 g</td>\n<td align="center" class="dosechart-method">oral</td>\n<td class="dosechart-substance"><a href="/plants/kratom/">Kratom</a></td>\n<td class="dosechart-form">(daily)</td>\n</tr>\n<tr>\n<td align="right" class="dosechart-time">\xa0 T+ 0:00</td>\n<td align="center" class="dosechart-amount">210 mg</td>\n<td align="center" class="dosechart-method">oral</td>\n<td class="dosechart-substance">Harmine</td>\n<td class="dosechart-form">(capsule)</td>\n</tr>\n<tr>\n<td align="right" class="dosechart-time">\xa0 T+ 0:00</td>\n<td align="center" class="dosechart-amount">210 mg</td>\n<td align="center" class="dosechart-method">oral</td>\n<td class="dosechart-substance"><a href="/chemicals/harmala/">Tetrahydroharmine<

In [5]:
#mapping scrapted tables html to schema
import pandas as pd
from bs4 import BeautifulSoup

schema_fields = [
    "previous_experience",
    "set_and_setting",
    "dose_amount", # Renamed dose_and_route to dose_amount
    "dose_method", # Added new field for dose method
    "before_after_changes",
    "extraordinary_thinking",
    "combination_with_other_substances",
    "intention",
    "experience_phases",
    "onset_description",
    "perceived_realness",
    "objective_elements",
    "entities_or_other_beings",
    "childhood_trauma",
    "time_relation",
    "sex_effects",
    "self_love_experience",
    "semantic_relations",
    "year_of_experience",
    "age",
    "gender",
    "substance_used",
    "most_important_element"
]

processed_reports = []

for report_data in extracted_data:
    processed_report = {}
    for field in schema_fields:
        processed_report[field] = None  # Initialize all fields to None

    # Map scraped data to schema_fields
    if report_data.get('substance'):
        processed_report['substance_used'] = report_data['substance']

    if report_data.get('bodyweight'):
        bodyweight_soup = BeautifulSoup(report_data['bodyweight'], 'html.parser')
        bodyweight_td = bodyweight_soup.find('td', class_='bodyweight-amount')
        processed_report['age'] = bodyweight_td.text.strip() if bodyweight_td else None

    if report_data.get('footdata'):
        footdata_soup = BeautifulSoup(report_data['footdata'], 'html.parser')

        year_td = footdata_soup.find('td', class_='footdata-expyear')
        processed_report['year_of_experience'] = year_td.text.replace('Exp Year:', '').strip() if year_td else None

        gender_td = footdata_soup.find('td', class_='footdata-gender')
        processed_report['gender'] = gender_td.text.replace('Gender:', '').strip() if gender_td else None

        age_td = footdata_soup.find('td', class_='footdata-ageofexp')
        processed_report['age'] = age_td.text.replace('Age at time of experience:', '').strip() if age_td else None

    # Extract dose amount and method from dosechart
    if report_data.get('dosechart'):
        dosechart_soup = BeautifulSoup(report_data['dosechart'], 'html.parser')
        amount_td = dosechart_soup.find('td', class_='dosechart-amount')
        method_td = dosechart_soup.find('td', class_='dosechart-method')

        processed_report['dose_amount'] = amount_td.text.strip() if amount_td else None
        processed_report['dose_method'] = method_td.text.strip() if method_td else None

    processed_reports.append(processed_report)

# Create a pandas DataFrame from the processed data
df_processed_reports = pd.DataFrame(processed_reports)

# Display the first few rows of the DataFrame
display(df_processed_reports.head())

,previous_experience,set_and_setting,dose_amount,dose_method,before_after_changes,extraordinary_thinking,combination_with_other_substances,intention,experience_phases,onset_description,...,childhood_trauma,time_relation,sex_effects,self_love_experience,semantic_relations,year_of_experience,age,gender,substance_used,most_important_element
0,None,None,1 - 3 g,oral,None,None,None,None,None,None,...,None,None,None,None,None,2024,36,Female,DMT Fumarate & Harmala Alkaloids,None
1,None,None,138 mg,smoked,None,None,None,None,None,None,...,None,None,None,None,None,2018,25,Female,DMT & Changa,None
2,None,None,75 mg,insufflated,None,None,None,None,None,None,...,None,None,None,None,None,2011,21,Male,DMT & Methoxetamine,None
3,None,None,3.0 g,oral,None,None,None,None,None,None,...,None,None,None,None,None,2007,Not Given,Male,"Syrian Rue, Cacti - T. peruvianus, 2C-E, 4-AcO...",None
4,None,None,,smoked,None,None,None,None,None,None,...,None,None,None,None,None,2007,Not Given,Male,DMT,None


In [6]:
#extracting reports html
import requests
from bs4 import BeautifulSoup
import json

# Assuming 'links' list is available from the previous scraping step

report_html_contents = []

for link in links:
    try:
        response = requests.get(link, verify=False) # Disable SSL verification
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        report_html_contents.append({
            'link': link,
            'html_content': response.text # Store text content instead of bytes
        })
        print(f"Fetched content for: {link}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching content for {link}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred while fetching {link}: {e}")

print(f"Finished fetching content for {len(report_html_contents)} reports.")

# Save the scraped data to a JSON file
with open('reports_html.json', 'w') as f:
    json.dump(report_html_contents, f, indent=4)

print("Scraped HTML data saved to reports_html.json")

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetched content for: https://www.erowid.org/experiences/exp.php?ID=118252


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetched content for: https://www.erowid.org/experiences/exp.php?ID=112397


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetched content for: https://www.erowid.org/experiences/exp.php?ID=90269


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetched content for: https://www.erowid.org/experiences/exp.php?ID=83098
Fetched content for: https://www.erowid.org/experiences/exp.php?ID=92381
Finished fetching content for 100 reports.
Scraped HTML data saved to reports_html.json


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
#cleaning reports html
from bs4 import BeautifulSoup, Comment # Import Comment to find HTML comments
import json

cleaned_reports = []

for report_data in report_html_contents:
    link = report_data['link']
    html_content = report_data['html_content']

    try:
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the div containing the report text
        report_div = soup.find('div', class_='report-text-surround')
        report_text = None

        if report_div:
            # The main report text is often between <!-- Start Body --> and <!-- End Body --> comments
            # within the 'report-text-surround' div. We need to iterate through the contents
            # to find the text nodes and handle <br> tags.
            content_started = False
            extracted_lines = []
            for content in report_div.contents:
                if isinstance(content, Comment) and "Start Body" in content:
                    content_started = True
                    continue
                if isinstance(content, Comment) and "End Body" in content:
                    content_started = False
                    break
                if content_started:
                    if content.name == 'br':
                        extracted_lines.append('\n') # Replace <br> with newline
                    elif isinstance(content, str):
                        extracted_lines.append(content)
                    # You might need to handle other tags within the text if they exist,
                    # but for simple text and line breaks, this should work.
                    # If there are nested tags with text, you might need a recursive approach
                    # or use get_text() on specific elements if they consistently contain text.
                    elif hasattr(content, 'get_text'):
                         extracted_lines.append(content.get_text())


            report_text = "".join(extracted_lines).strip()

        if report_text:
            cleaned_reports.append({
                'link': link,
                'report_text': report_text
            })
            print(f"Successfully extracted report text for {link}")
        else:
            print(f"Warning: Could not extract report text for {link}")

    except Exception as e:
        print(f"Error processing HTML for {link}: {e}")

print(f"Finished extracting report text for {len(cleaned_reports)} reports.")

# Save the cleaned reports to a JSON file
with open('cleaned_reports.json', 'w') as f:
    json.dump(cleaned_reports, f, indent=4)

print("Cleaned report data saved to cleaned_reports.json")

Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=118252
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=112397
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=96444
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=64048
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=62835
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=34866
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=113441
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=113133
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=99695
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=94632
Successfully extracted report text for https://www.erowid.org/experiences/exp.php?ID=1